In [ ]:
from utils.cn import Linear
from utils.tensor import Tensor
import numpy as np

m = Linear(n_in=600, n_out=600, bias=True)

x = Tensor(np.random.randn(10, 600))

y = m(x)
y.backward()

print(y)
print(m.parameter.grad)
print(m.bias.grad)

In [ ]:
import numpy as np

arr = np.zeros([2,3,4,5,6])
arr = np.array([12])

shape = arr.shape

# a = shape[:2]
# b = shape[2:4]
# c = shape[4:]
# f = a + b + c
# f
shape[:0], shape[0], shape[0+1:]

In [ ]:
def flatten(arr, start_dim: int = 0, end_dim: int = -1):
    shape = arr.shape

    if start_dim < 0:
        start_dim = len(shape) + start_dim 
    if end_dim < 0:
        end_dim = len(shape) + end_dim + 1
    
    # Compose new shape
    start_shape = shape[:start_dim]
    flatten_dim = np.prod(shape[start_dim:end_dim])
    end_shape = shape[end_dim:]


    final_shape = start_shape + (flatten_dim,) + end_shape

    flat = np.reshape(arr, final_shape)
    return flat

shaped = flatten(arr, 1, -1)
print(shaped.shape)

b = shaped.reshape(shape)
b.shape

In [ ]:
import numpy as np
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(arr.size)


In [ ]:
np.append(np.ones(3), np.ones(3)).reshape(2,3).T

In [ ]:
import numpy as np

#Im2Col

C = 2
im = np.arange(C*5*5).reshape((C,5,5))
kernel_size = (3,3)
print(im)
print("\n")

# Calculate output size
h_out = im.shape[1] - kernel_size[0] + 1
w_out = im.shape[2] - kernel_size[1] + 1

# Use stride tricks to create a sliding window view
# The shape will be (d, h_out, w_out, kH, kW)
shape = (C, h_out, w_out, kernel_size[0], kernel_size[1])
strides = (
    im.strides[0],
    im.strides[1],
    im.strides[2],
    im.strides[1],
    im.strides[2]
)
windows = np.lib.stride_tricks.as_strided(im, shape=shape, strides=strides)

# Move channel to the last dim and reshape to a 2D array as desired
col = windows.transpose(1, 2, 0, 3, 4).reshape(h_out * w_out, C * kernel_size[0] * kernel_size[1]).T
col

In [ ]:
import numpy as np

# This is still roughly 10-20 times slower than using np.strides from above, but I refuse to use that implementation
## since this part of the code now takes up relatively very little time (I hope)
## It might also eat up all the memory of the computer. You never know. Seems to scale fine with input dim but less fine wiht spacial img size
## The stride code is disgusting

# It might be smarter to loop over rows instead of columns? It is symmetrical along the diagonal within each image and dim after all
## Looping over rows: kernel_size * input_dim
## Looping over cols: output_pixels * batch_size

d = 2
im = np.arange(d*5*5).reshape((d,5,5))
start_pos = (1,2)
kernel_size = (3,3)
print(im)
print("\n")

h = list(range(im.shape[1] - kernel_size[0] + 1))
w = list(range(im.shape[2] - kernel_size[1] + 1))

col = []
for i in h:
    for j in w:
        patch = im[:, i:i + kernel_size[0], j:j + kernel_size[1]]
        reshaped = patch.reshape(-1)
        col.append(reshaped) #Using Python append and not np.append (O(1) vs O(N^2))

col = np.array(col).T
print(col)

In [45]:
import numpy as np

C = 1
N = 1
H = 5
W = 5
KH = 3
KW = 3
stride = 2
padding = 0
im = np.arange(N*C*H*W).reshape((N,C,H,W))
start_pos = (1,2)
print(im)
print("\n")

h_out = int((H + 2 * padding - KH) // stride + 1)
w_out = int((W + 2 * padding - KW) // stride + 1)
print(h_out, w_out)

col = []
for n in range(N):
    for i in range(0, H + 2*padding - KH + 1, stride):
        for j in range(0, W + 2*padding - KW + 1, stride):
            patch = im[n, :, i:i+KH, j:j+KW]
            # Flatten patch to (C * KH * KW)
            col.append(patch.reshape(-1)) #Using Python append and not np.append (O(1) vs O(N^2))

col = np.array(col).T
print(col)


[[[[ 0  1  2  3  4]
   [ 5  6  7  8  9]
   [10 11 12 13 14]
   [15 16 17 18 19]
   [20 21 22 23 24]]]]


2 2
[[ 0  2 10 12]
 [ 1  3 11 13]
 [ 2  4 12 14]
 [ 5  7 15 17]
 [ 6  8 16 18]
 [ 7  9 17 19]
 [10 12 20 22]
 [11 13 21 23]
 [12 14 22 24]]


In [ ]:
grad = np.array(col)
# (C*KH*KW, N*h_out*w_out), split dim 0 into (C, KH*KW), split dim 1 into (N, h_out*w_out)
grad = grad.reshape(C, KH * KW, N, h_out * w_out)
grad = grad.transpose(2, 0, 1, 3) # permute to (N, C, KH*KW, h_out_w_out)

grad_out = np.zeros((N, C, H*W))

#TODO: remove padding gradients

if stride != 1: # if stride, we pad the matrix with 0s such that we get the same shape as if stride=1
    grad = grad.reshape(N, C, KH * KW, h_out, w_out)
    # dims as if stride=1
    h_stride1 = H + 2 * padding - KH + 1
    w_stride1 = W + 2 * padding - KW + 1
    grad_dilated = np.zeros((N, C, KH * KW, h_stride1, w_stride1), dtype=grad.dtype)
    grad_dilated[:, :, :, ::stride, ::stride] = grad
    grad = grad_dilated.reshape(N, C, KH * KW, -1)
    # now the grad matrix is back to this format, we update the spacials dims
    h_out = h_stride1
    w_out = w_stride1

print(grad[0][0])


for i in range(W - w_out): # Pad with 0s between each downward kernel move to have axis=3 follow a flattened pattern (W*h_out)-(W - w_out)
    where = [((j+1)*(w_out) + i*(j+1)) for j in range(h_out-1)]
    grad = np.insert(grad, where, 0, axis=3)

print(grad[0][0])

for r in range(KH*KW):
    start = r % KW + W * (r//KW)
    end = W * h_out + r % KW + W * (r//KW) - (W-w_out)
    print(start,end)
    grad_out[:, :, start:end] += grad[:, :, r, :]

print(grad_out[0][0])


[[ 0  0  2  0  0  0 10  0 12]
 [ 1  0  3  0  0  0 11  0 13]
 [ 2  0  4  0  0  0 12  0 14]
 [ 5  0  7  0  0  0 15  0 17]
 [ 6  0  8  0  0  0 16  0 18]
 [ 7  0  9  0  0  0 17  0 19]
 [10  0 12  0  0  0 20  0 22]
 [11  0 13  0  0  0 21  0 23]
 [12  0 14  0  0  0 22  0 24]]
[[ 0  0  2  0  0  0  0  0  0  0 10  0 12]
 [ 1  0  3  0  0  0  0  0  0  0 11  0 13]
 [ 2  0  4  0  0  0  0  0  0  0 12  0 14]
 [ 5  0  7  0  0  0  0  0  0  0 15  0 17]
 [ 6  0  8  0  0  0  0  0  0  0 16  0 18]
 [ 7  0  9  0  0  0  0  0  0  0 17  0 19]
 [10  0 12  0  0  0  0  0  0  0 20  0 22]
 [11  0 13  0  0  0  0  0  0  0 21  0 23]
 [12  0 14  0  0  0  0  0  0  0 22  0 24]]
0 13
1 14
2 15
5 18
6 19
7 20
10 23
11 24
12 25
[ 0.  1.  4.  3.  4.  5.  6. 14.  8.  9. 20. 22. 48. 26. 28. 15. 16. 34.
 18. 19. 20. 21. 44. 23. 24.]


In [16]:
where = [i +(j+1)*w_out for j in range(0,h_out,stride) for i in range(w_out)]
where

[2, 3]

In [98]:
a = np.arange(5*7*3).reshape(3,5,7)
for i in range(2):
    a = np.insert(a, (1+i,3+i*2), 0, axis=2)
a
# for i in range(a.shape[1]//3):
    

array([[[  0,   0,   0,   1,   2,   0,   0,   3,   4,   5,   6],
        [  7,   0,   0,   8,   9,   0,   0,  10,  11,  12,  13],
        [ 14,   0,   0,  15,  16,   0,   0,  17,  18,  19,  20],
        [ 21,   0,   0,  22,  23,   0,   0,  24,  25,  26,  27],
        [ 28,   0,   0,  29,  30,   0,   0,  31,  32,  33,  34]],

       [[ 35,   0,   0,  36,  37,   0,   0,  38,  39,  40,  41],
        [ 42,   0,   0,  43,  44,   0,   0,  45,  46,  47,  48],
        [ 49,   0,   0,  50,  51,   0,   0,  52,  53,  54,  55],
        [ 56,   0,   0,  57,  58,   0,   0,  59,  60,  61,  62],
        [ 63,   0,   0,  64,  65,   0,   0,  66,  67,  68,  69]],

       [[ 70,   0,   0,  71,  72,   0,   0,  73,  74,  75,  76],
        [ 77,   0,   0,  78,  79,   0,   0,  80,  81,  82,  83],
        [ 84,   0,   0,  85,  86,   0,   0,  87,  88,  89,  90],
        [ 91,   0,   0,  92,  93,   0,   0,  94,  95,  96,  97],
        [ 98,   0,   0,  99, 100,   0,   0, 101, 102, 103, 104]]])

In [ ]:
from utils.dataloader import DataLoader, train_dataset, test_dataset
batch_size = 34
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=False)

len(test_loader.dataset)
test_loader.dataset.y

In [ ]:
from utils import candle

candle.is_grad_enabled()

with candle.no_grad():
    print(candle.is_grad_enabled())

In [ ]:
import numpy as np

b = np.array([[1, 2, 3], [4, 5, 6]])
f = np.array([[7, 8], [9, 10]])

b.T @ f

In [ ]:
False and True

In [ ]:
if not False:
    print("False")

In [ ]:
(f.T @ b).T

In [ ]:
dict_ = {"side": "test"}

try:
    start_index = dict_["start_index"]
    end_index = dict_["end_index"]
    matrix_side = dict_["matrix"]
except:
    start_index = None
    end_index = None
    matrix_side = None

side = dict_["side"]

if "matrix" in dict_:
    print("test1")

if 2 != None:
    print("test2")

In [ ]:
import numpy as np

# Create two 3D matrices (tensors)
# For valid matrix multiplication with shape (a, b, c) @ (a, c, d) -> (a, b, d)
A = np.random.randint(0, 10, (2, 3, 4))
N = np.random.randint(0, 10, (2, 4, 5))

# Multiply each (i)th 3x4 with (i)th 4x5 -> each result (i) is 3x5. All dims used.
C = np.empty((2, 3, 5), dtype=int)
for i in range(A.shape[0]):
    C[i] = A[i] @ N[i]

print("A.shape:", A.shape)
print("B.shape:", N.shape)
print("C.shape:", C.shape)
print(C)
